In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from cgmap.mapping.mapper import Mapper

args_dict = {
    'mapping': 'martini3',
    'input': '/scratch/angiod/nmr/ucbshift.single/test/rcsb.pdb/',
    # 'inputtraj': '/protein.trr',
    # 'traj_format': 'trr',
    # 'trajslice': ':10',
    'selection': 'protein',
    'output': '/scratch/angiod/nmr/ucbshift.single/test/rcsb.CG.npz/',
    # 'outputtraj': 'trr',
    'isatomistic': True,
}

mapping = Mapper(args_dict=args_dict)

# --- Option 1 --- #
# Map only the ith file in the list of files (and optionally trajectories)

# mapping.map(index=0)
# mapping.save(filename='protein.CG.pdb')
# mapping.save_atomistic()
# mapping.save_npz(filename='/my/path/to/protein.npz', to_pos_unit='Angstrom', to_force_unit='kcal/(mol*Angstrom)')

# --- Option 2 --- #
# Map all the files one after the other

for i, (m, npz_filename) in enumerate(mapping()):
#     m.save()
#     m.save_atomistic()
    m.save_npz(filename=npz_filename, from_pos_unit='Angstrom', to_pos_unit='Angstrom', from_force_unit='kcal/(mol*Angstrom)', to_force_unit='kcal/(mol*Angstrom)')